In [8]:
import tensorflow as tf
from datetime import datetime
import math
import time

slim=tf.contrib.slim
'''
相当于把tf.truncated_normal_initializer重新包装了一层
默认生成均值为0.0，标准差为输入值的初始化器
'''
trunc_normal = lambda stddev:tf.truncated_normal_initializer(0.0,stddev)

In [14]:
'''
生成默认参数
tf.GraphKeys.UPDATE_OPS的目的：
https://stackoverflow.com/questions/48260394/whats-the-differences-between-tf-graphkeys-trainable-variables-and-tf-graphkeys
'''
def inception_v3_arg_scope(weight_decay=0.00004, # L2正则项的weight-decay，详见weight-decay具体含义
                           stddev=0.1,
                           batch_norm_var_collection='moving_vars'):
    batch_norm_params = {
        'decay':0.9997,
        'epsilon':0.001,
        'updates_collections':tf.GraphKeys.UPDATE_OPS, # 这个是啥 拿到图中所有需要被更新的ops的名称,？
        'variables_collections':{
            'beta':None,
            'gamma':None,
            'moving_mean':[batch_norm_var_collection],
            'moving_variance':[batch_norm_var_collection],
        }
    }
    with slim.arg_scope([slim.conv2d,slim.fully_connected],
                        weights_regularizer=slim.l2_regularizer(weight_decay)):
        with slim.arg_scope([slim.conv2d],
                            weights_initializer=tf.truncated_normal_initializer(stddev=stddev),
                            activation_fn=tf.nn.relu,
                            normalizer_params = batch_norm_params
                           ) as sc:
            return sc

In [58]:
def inception_v3_base(inputs,scope=None):
    # 保存关键节点
    end_points = {}
    with tf.variable_scope(scope,'InceptionV3_3',[inputs]):
        '''
            设置卷积/最大池化/平均池化的默认步长为1,padding模式为VALID
            即，设置超参数
            
            算是设置了初始的几层卷积层
        '''
        with slim.arg_scope([slim.conv2d,
                             slim.max_pool2d,
                             slim.avg_pool2d],
                             stride=1,
                             padding='VALID'):
            net = slim.conv2d(inputs,32,[3,3],stride=2,scope='Conv2d_1a_3x3_1')
            net = slim.conv2d(net,32,[3,3],scope='Conv2d_2a_3x3_1')
            net = slim.conv2d(net,64,[3,3],padding='SAME',scope='Conv2d_2b_3x3_1')
            
            net = slim.max_pool2d(net, [3, 3], stride=2, scope='MaxPool_3a_3x3_1')
            net = slim.conv2d(net, 80, [1, 1], scope='Conv2d_3b_1x1_1')
            net = slim.conv2d(net, 192, [3, 3], scope='Conv2d_4a_3x3_1')
            net = slim.max_pool2d(net, [3, 3], stride=2, scope='MaxPool_5a_3x3_1')
        
        '''
            设置卷积/最大池化/平均池化的默认步长为1,padding模式为SAME
        '''
        with slim.arg_scope([slim.conv2d,slim.max_pool2d,slim.avg_pool2d],
                           stride=1,
                           padding='SAME'):
            '''
                第一个inception模组的第一个inception块
            '''
            with tf.variable_scope('Mixed_5b'):
                # branch 1：1*1*64
                with tf.variable_scope('Branch_0'):
                    branch_0 = slim.conv2d(net,64,[1,1],scope='Conv2d_0a_1x1')
                # branch 2: 1*1*48 接5*5*64
                with tf.variable_scope('Branch_1'):
                    branch_1 = slim.conv2d(net,48,[1,1],scope='Conv2d_0a_1x1')
                    branch_1 = slim.conv2d(branch_1,64,[5,5],scope='Conv2d_0b_5x5')
                # branch 3: 1*1*64 接 3*3*96 3*3*96
                with tf.variable_scope('Branch_2'):
                    branch_2 = slim.conv2d(net,64,[1,1],scope='Conv2d_0a_1x1')
                    branch_2 = slim.conv2d(branch_2,96,[3,3],scope='Conv2d_0b_3x3')
                    branch_2 = slim.conv2d(branch_2,96,[3,3],scope='Conv2d_0c_3x3')
                # branch 4: 3*3 average pooling,1*1*32
                with tf.variable_scope('Branch_3'):
                    branch_3 = slim.avg_pool2d(net,[3,3],scope='AvgPool_0a_3x3')
                    branch_3 = slim.conv2d(branch_3,32,[1,1],scope='Conv2d_0b_1x1')
                # concate all the 4 branches
                net = tf.concat([branch_0,branch_1,branch_2,branch_3],axis=3)
            '''
                第一个inception模组的第二个inception块
            '''
            with tf.variable_scope('Mixed_5c'):
                # branch 1:1*1*64
                with tf.variable_scope('Branch_0'):
                    branch_0 = slim.conv2d(net,64,[1,1],scope='Conv2d_0a_1x1')
                # branch 2:1*1*48,5*5*64
                with tf.variable_scope('Branch_1'):
                    branch_1 = slim.conv2d(net,48,[1,1],scope='Conv2d_0a_1x1')
                    branch_1 = slim.conv2d(branch_1,64,[5,5],scope='Conv2d_0b_5x5')
                # branch 3:1*1*64,3*3*96 3*3*96
                with tf.variable_scope('Branch_2'):
                    branch_2 = slim.conv2d(net,64,[1,1],scope='Conv2d_0a_1x1')
                    branch_2 = slim.conv2d(branch_2,96,[3,3],scope='Conv2d_0b_3x3')
                    branch_2 = slim.conv2d(branch_2,96,[3,3],scope='Conv2d_0c_3x3')
                # branch 4: 3*3 avg pooling 1*1*64
                with tf.variable_scope('Branch_3'):
                    branch_3 = slim.avg_pool2d(net,[3,3],scope='AvgPool_0a_3x3')
                    branch_3 = slim.conv2d(branch_3,64,[1,1],scope='Conv2d_0b_1x1')
                # concate all the 4 branches
                net = tf.concat([branch_0,branch_1,branch_2,branch_3],axis=3)
            '''
                第一个inception模组的第三个inception块
            '''
            with tf.variable_scope('Mixed_5d'):
                # branch 1:1*1*64
                with tf.variable_scope('Branch_0'):
                    branch_0 = slim.conv2d(net,64,[1,1],scope='Conv2d_0a_1x1')
                # branch 2:1*1*48,5*5*64
                with tf.variable_scope('Branch_1'):
                    branch_1 = slim.conv2d(net,48,[1,1],scope='Conv2d_0a_1x1')
                    branch_1 = slim.conv2d(branch_1,64,[5,5],scope='Conv2d_0b_5x5')
                # branch 3:1*1*64,3*3*96 3*3*96
                with tf.variable_scope('Branch_2'):
                    branch_2 = slim.conv2d(net,64,[1,1],scope='Conv2d_0a_1x1')
                    branch_2 = slim.conv2d(branch_2,96,[3,3],scope='Conv2d_0b_3x3')
                    branch_2 = slim.conv2d(branch_2,96,[3,3],scope='Conv2d_0c_3x3')
                # branch 4: 3*3 avg pooling 1*1*64
                with tf.variable_scope('Branch_3'):
                    branch_3 = slim.avg_pool2d(net,[3,3],scope='AvgPool_0a_3x3')
                    branch_3 = slim.conv2d(branch_3,64,[1,1],scope='Conv2d_0b_1x1')
                # concate all the 4 branches
                net = tf.concat([branch_0,branch_1,branch_2,branch_3],axis=3)
            
            '''
                第二个inception模组的预分支
            '''
            with tf.variable_scope('Mixed_6a'):
                # branch 1 : 3*3*384,strides=2,padding=valid
                with tf.variable_scope('Branch_0'):
                    branch_0 = slim.conv2d(net,384,[3,3],stride=2,padding='VALID',scope='Conv2d_1a_1x1')
                # branch 2 : 1*1*64,3*3*96 *2
                with tf.variable_scope('Branch_1'):
                    branch_1 = slim.conv2d(net,64,[1,1],scope='Conv2d_1a_1x1')
                    branch_1 = slim.conv2d(branch_1,96,[3,3],scope='Conv2d_1b_3x3')
                    # strid=2,padding=valid
                    branch_1 = slim.conv2d(branch_1,96,[3,3],stride=2,padding='VALID',scope='Conv2d_1c_3x3')
                # branch 3 : 3*3 max pool strid=2,padding=valid;
                with tf.variable_scope('Branch_2'):
                    branch_2 = slim.max_pool2d(net,[3,3],stride=2,padding='VALID',scope='MaxPool_1a_3x3')
                # concate all 3 branches
                net = tf.concat([branch_0,branch_1,branch_2],axis=3)
            '''
                第二个inception模组的第二个inception块
            '''
            with tf.variable_scope('Mixed_6b'):
                # branch 1 1*1*192
                with tf.variable_scope('Branch_0'):
                    branch_0 = slim.conv2d(net,192,[1,1],scope='Conv2d_1a_1x1')
                # branch 2 1*1*128,1*7*128,7*1*192
                with tf.variable_scope('Branch_1'):
                    branch_1 = slim.conv2d(net,128,[1,1],scope='Conv2d_1a_1x1')
                    branch_1 = slim.conv2d(branch_1,128,[1,7],scope='Conv2d_1b_1x7')
                    branch_1 = slim.conv2d(branch_1,192,[7,1],scope='Conv2d_1c_7x1')
                # branch 3: 1*1*128 [(7,1)*128,(1,7)*128,(7,1)*128,(1,7)*192]
                with tf.variable_scope('Branch_2'):
                    branch_2 = slim.conv2d(net,128,[1,1],scope='Conv2d_1a_1x1')
                    branch_2 = slim.conv2d(branch_2,128,[7,1],scope='Conv2d_1b_7x1')
                    branch_2 = slim.conv2d(branch_2,128,[1,7],scope='Conv2d_1c_1x7')
                    branch_2 = slim.conv2d(branch_2,128,[7,1],scope='Conv2d_1d_7x1')
                    branch_2 = slim.conv2d(branch_2,192,[1,7],scope='Conv2d_1e_1x7')
                # branch 4: 3*3 avr pooling;1*1*192
                with tf.variable_scope('Branch_3'):
                    branch_3 = slim.avg_pool2d(net,[3,3],scope='AvgPool_1a_3x3')
                    branch_3 = slim.conv2d(branch_3,192,[1,1],scope='Conv2d_1b_1x1')
                # concate all 4 branches
                net = tf.concat([branch_0,branch_1,branch_2,branch_3],axis=3)
            '''
                第二个inception模组的第三个inception块
            '''
            with tf.variable_scope('Mixed_6c'):
                # branch 1 1*1*192
                with tf.variable_scope('Branch_0'):
                    branch_0 = slim.conv2d(net,192,[1,1],scope='Conv2d_1a_1x1')
                # branch 2 1*1*128,1*7*128,7*1*192
                with tf.variable_scope('Branch_1'):
                    branch_1 = slim.conv2d(net,160,[1,1],scope='Conv2d_1a_1x1')
                    branch_1 = slim.conv2d(branch_1,160,[1,7],scope='Conv2d_1b_1x7')
                    branch_1 = slim.conv2d(branch_1,192,[7,1],scope='Conv2d_1c_7x1')
                # branch 3: 1*1*128 [(7,1)*128,(1,7)*128,(7,1)*128,(1,7)*192]
                with tf.variable_scope('Branch_2'):
                    branch_2 = slim.conv2d(net,160,[1,1],scope='Conv2d_1a_1x1')
                    branch_2 = slim.conv2d(branch_2,160,[7,1],scope='Conv2d_1b_7x1')
                    branch_2 = slim.conv2d(branch_2,160,[1,7],scope='Conv2d_1c_1x7')
                    branch_2 = slim.conv2d(branch_2,160,[7,1],scope='Conv2d_1d_7x1')
                    branch_2 = slim.conv2d(branch_2,192,[1,7],scope='Conv2d_1e_1x7')
                # branch 4: 3*3 avr pooling;1*1*192
                with tf.variable_scope('Branch_3'):
                    branch_3 = slim.avg_pool2d(net,[3,3],scope='AvgPool_1a_3x3')
                    branch_3 = slim.conv2d(branch_3,192,[1,1],scope='Conv2d_1b_1x1')
                # concate all 4 branches
                net = tf.concat([branch_0,branch_1,branch_2,branch_3],axis=3)
            
            '''
                第二个inception模组的第四个inception块
            '''
            with tf.variable_scope('Mixed_6d'):
                # branch 1 1*1*192
                with tf.variable_scope('Branch_0'):
                    branch_0 = slim.conv2d(net,192,[1,1],scope='Conv2d_1a_1x1')
                # branch 2 1*1*128,1*7*128,7*1*192
                with tf.variable_scope('Branch_1'):
                    branch_1 = slim.conv2d(net,160,[1,1],scope='Conv2d_1a_1x1')
                    branch_1 = slim.conv2d(branch_1,160,[1,7],scope='Conv2d_1b_1x7')
                    branch_1 = slim.conv2d(branch_1,192,[7,1],scope='Conv2d_1c_7x1')
                # branch 3: 1*1*128 [(7,1)*128,(1,7)*128,(7,1)*128,(1,7)*192]
                with tf.variable_scope('Branch_2'):
                    branch_2 = slim.conv2d(net,160,[1,1],scope='Conv2d_1a_1x1')
                    branch_2 = slim.conv2d(branch_2,160,[7,1],scope='Conv2d_1b_7x1')
                    branch_2 = slim.conv2d(branch_2,160,[1,7],scope='Conv2d_1c_1x7')
                    branch_2 = slim.conv2d(branch_2,160,[7,1],scope='Conv2d_1d_7x1')
                    branch_2 = slim.conv2d(branch_2,192,[1,7],scope='Conv2d_1e_1x7')
                # branch 4: 3*3 avr pooling;1*1*192
                with tf.variable_scope('Branch_3'):
                    branch_3 = slim.avg_pool2d(net,[3,3],scope='AvgPool_1a_3x3')
                    branch_3 = slim.conv2d(branch_3,192,[1,1],scope='Conv2d_1b_1x1')
                # concate all 4 branches
                net = tf.concat([branch_0,branch_1,branch_2,branch_3],axis=3)
            
            '''
                第二个inception模组的第五个inception块
            '''
            with tf.variable_scope('Mixed_6e'):
                # branch 1 1*1*192
                with tf.variable_scope('Branch_0'):
                    branch_0 = slim.conv2d(net,192,[1,1],scope='Conv2d_1a_1x1')
                # branch 2 1*1*128,1*7*128,7*1*192
                with tf.variable_scope('Branch_1'):
                    branch_1 = slim.conv2d(net,192,[1,1],scope='Conv2d_1a_1x1')
                    branch_1 = slim.conv2d(branch_1,192,[1,7],scope='Conv2d_1b_1x7')
                    branch_1 = slim.conv2d(branch_1,192,[7,1],scope='Conv2d_1c_7x1')
                # branch 3: 1*1*128 [(7,1)*128,(1,7)*128,(7,1)*128,(1,7)*192]
                with tf.variable_scope('Branch_2'):
                    branch_2 = slim.conv2d(net,192,[1,1],scope='Conv2d_1a_1x1')
                    branch_2 = slim.conv2d(branch_2,192,[7,1],scope='Conv2d_1b_7x1')
                    branch_2 = slim.conv2d(branch_2,192,[1,7],scope='Conv2d_1c_1x7')
                    branch_2 = slim.conv2d(branch_2,192,[7,1],scope='Conv2d_1d_7x1')
                    branch_2 = slim.conv2d(branch_2,192,[1,7],scope='Conv2d_1e_1x7')
                # branch 4: 3*3 avr pooling;1*1*192
                with tf.variable_scope('Branch_3'):
                    branch_3 = slim.avg_pool2d(net,[3,3],scope='AvgPool_1a_3x3')
                    branch_3 = slim.conv2d(branch_3,192,[1,1],scope='Conv2d_1b_1x1')
                # concate all 4 branches
                net = tf.concat([branch_0,branch_1,branch_2,branch_3],axis=3)
            
            # 保存第二个inception模组的最后一个输出
            end_points['Mixed_6e']=net
            
            '''
                第三个inception模组的第一个inception块
            '''
            with tf.variable_scope('Mixed_7a'):
                # branch 1:1*1*192,3*3*320,strides=2
                with tf.variable_scope('Branch_0'):
                    branch_0 = slim.conv2d(net,192,[1,1],scope='Conv2d_2a_1x1')
                    branch_0 = slim.conv2d(branch_0,320,[3,3],stride=2,padding='VALID',scope='Conv2d_2b_3x3')
                # branch 2: 1*1*192,1*7*192,7*1*192,3*3*192 strides=2
                with tf.variable_scope('Branch_1'):
                    branch_1 = slim.conv2d(net,192,[1,1],scope='Conv2d_2a_1x1')
                    branch_1 = slim.conv2d(branch_1,192,[1,7],scope='Conv2d_2b_1x7')
                    branch_1 = slim.conv2d(branch_1,192,[7,1],scope='Conv2d_2c_7x1')
                    branch_1 = slim.conv2d(branch_1,192,[3,3],stride=2,padding='VALID',scope='Conv2d_2d_3x3')
                # branch 3: 3*3 maxpool
                with tf.variable_scope('Branch_2'):
                    branch_2 = slim.max_pool2d(net,[3,3],stride=2,padding='VALID',scope='MaxPool_2a_3x3')
                # concate all 3 branch
                net = tf.concat([branch_0,branch_1,branch_2],3)
            
            '''
                第三个inception模组的第二个inception块
            '''
            with tf.variable_scope('Mixed_7b'):
                # branch_1 : 1*1*320
                with tf.variable_scope('Branch_0'):
                    branch_0 = slim.conv2d(net,320,[1,1],scope='Conv2d_3a_1x1')
                # branch_2 : 1*1*384 [1*3*384,3*1*384]
                with tf.variable_scope('Branch_1'):
                    branch_1 = slim.conv2d(net,384,[1,1],scope='Conv2d_3a_1x1')
                    branch_1 = tf.concat([slim.conv2d(branch_1,384,[1,3],scope='Conv2d_3a_1x3'),
                                          slim.conv2d(branch_1,384,[3,1],scope='Conv2d_3a_3x1')],
                                         axis=3)
                # branch_3 : 1*1*448,3*3*384, [1*3*384,3*1*384]
                with tf.variable_scope('Branch_2'):
                    branch_2 = slim.conv2d(net,448,[1,1],scope='Conv2d_3a_1x1')
                    branch_2 = slim.conv2d(branch_2,384,[3,3],scope='Conv2d_3b_3x3')
                    branch_2 = tf.concat([slim.conv2d(branch_2,384,[1,3],scope='Conv2d_3c_3x3'),
                                          slim.conv2d(branch_2,384,[3,2],scope='Conv2d_3d_3x3')],
                                         axis=3)
                # branch_4 : 3*3 avr pooling,1*1*192
                with tf.variable_scope('Branch_3'):
                    branch_3 = slim.avg_pool2d(net,[3,3],scope='AvgPool_3a_3x3')
                    branch_3 = slim.conv2d(branch_3,192,[1,1],scope='Conv2d_3b_1x1')
                
                net = tf.concat([branch_0,branch_1,branch_2,branch_3],axis=3)
                
            '''
                第三个inception模组的第三个inception块
            '''
            with tf.variable_scope('Mixed_7c'):
                # branch 1: 1*1*320
                with tf.variable_scope('Branch_0'):
                    branch_0 = slim.conv2d(net,320,[1,1],scope='Conv2d_4a_1x1')
                # branch 2: 1*1*384 [(1*3*384),(3,1,384)]
                with tf.variable_scope('Branch_1'):
                    branch_1 = slim.conv2d(net,384,[1,1],scope='Conv2d_4a_1x1')
                    branch_1 = tf.concat([slim.conv2d(branch_1,384,[1,3],scope='Conv2d_4b_1x3'),
                                          slim.conv2d(branch_1,384,[3,1],scope='Conv2d_4c_3x1')],
                                          axis=3)
                # branch_3 : 1*1*448,3*3*384, [(1*3),(3*1)]*384
                with tf.variable_scope('Branch_2'):
                    branch_2 = slim.conv2d(net,448,[1,1],scope='Conv2d_4a_1x1')
                    branch_2 = slim.conv2d(branch_2,384,[3,3],scope='Conv2d_4b_3x3')
                    branch_2 = tf.concat([slim.conv2d(branch_2,384,[1,3],scope='Conv2d_4c_1x3'),
                                          slim.conv2d(branch_2,384,[3,1],scope='Conv2d_4d_3x1')],
                                          axis=3)
                # branch 4 ：3x3 avr pool;1x1x192
                with tf.variable_scope('Branch_3'):
                    branch_3 = slim.avg_pool2d(net,[3,3],scope='AvgPool_4a_3x3')
                    branch_3 = slim.conv2d(branch_3,192,[1,1],scope='Conv2d_4b_1x1')
                # concate all 4 branches
                net = tf.concat([branch_0,branch_1,branch_2,branch_3],axis=3)
            
            return net,end_points
                    
                
                
        

            

In [61]:
def inception_v3(inputs,
                 num_classes=1000,
                 is_training=True,
                 dropout_keep_prob=0.8,
                 prediction_fn=slim.softmax,
                 spatial_squeeze=True,
                 reuse=None,
                 scope='InceptionV3'):
    with tf.variable_scope(scope,'InceptionV3',[inputs,num_classes],reuse=reuse) as scope:
        with slim.arg_scope([slim.batch_norm,slim.dropout],is_training=is_training):
            net,end_points=inception_v3_base(inputs,scope=scope)
            '''
              超参数定义
              辅助分类节点
            '''
            with slim.arg_scope([slim.conv2d,slim.max_pool2d,slim.avg_pool2d],stride=1,padding='SAME'):
                aux_logits = end_points['Mixed_6e']
                
                with tf.variable_scope('AuxLogits'):
                    aux_logits = slim.avg_pool2d(aux_logits,[5,5],stride=3,padding='VALID',scope='AvgPool_1a_5x5')
                    aux_logits = slim.conv2d(aux_logits,128,[1,1],scope='Conv2d_1b_1x1')
                    aux_logits = slim.conv2d(aux_logits,768,[5,5],weights_initializer=trunc_normal(0.01),padding='VALID',scope='Conv2d_2a_5x5')
                    aux_logits = slim.conv2d(aux_logits,num_classes,[1,1],activation_fn=None,
                                             normalizer_fn=None,weights_initializer=trunc_normal(0.001),scope='Conv2d_2b_1x1')
                    if spatial_squeeze:
                        aux_logits = tf.squeeze(aux_logits,[1,2],name='SpatialSqueeze')
                    end_points['AuxLogits'] = aux_logits
            '''
                正常分类
            '''
            with tf.variable_scope('Logits'):
                net = slim.avg_pool2d(net,[8,8],padding='VALID',scope='AvgPool_1a_8x8')
                net = slim.dropout(net,keep_prob=dropout_keep_prob,scope='Dropout_1b')
                end_points['PreLogits'] = net
                logits = slim.conv2d(net,num_classes,[1,1],activation_fn=None,
                                     normalizer_fn=None,scope='Conv2d_1c_1x1')
                if spatial_squeeze:
                    logits = tf.squeeze(logits,[1,2],name='SpatialSqueeze')
            
            # 保存各种节点
            end_points['Logits'] = logits
            end_points['Predictions'] = prediction_fn(logits,scope='Predictions')
    return logits,end_points
            

In [62]:
def time_tensorflow_run(session,target,info_string,num_batches):
    num_steps_burn_in = 10
    total_duration = 0.0
    total_duration_squared=0.0
    
    for i in range(num_batches + num_steps_burn_in):
        start_time = time.time()
        _ = session.run(target)
        # 持续时间
        duration = time.time()-start_time
        if i >= num_steps_burn_in:
            if not i % 10 :
                print('%s: step %d, duration = %.3f' % (datetime.now().strftime('%X'), 
                                                        i - num_steps_burn_in, 
                                                        duration))
            total_duration += duration
            total_duration_squared += duration*duration
    
    # 计算平均耗时
    mn = total_duration / num_batches
    vr = total_duration_squared / num_batches-mn*mn
    sd = math.sqrt(vr)
    
    print('%s: %s across %d steps, %.3f +/- %.3f sec / batch' % (datetime.now().strftime('%X'),
                                                                 info_string, 
                                                                 num_batches, 
                                                                 mn, sd))

In [64]:
log_dir='./logs/example5/'
tf.reset_default_graph()
batch_size = 32
height,width = 299,299
inputs = tf.random_uniform([batch_size,height,width,3])

with slim.arg_scope(inception_v3_arg_scope()):
    logits,end_points = inception_v3(inputs,is_training=False)

init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

writer = tf.summary.FileWriter(log_dir, sess.graph)


num_batches = 100
time_tensorflow_run(sess,logits,'Forward',num_batches)

16:11:02: step 0, duration = 4.155
16:11:44: step 10, duration = 4.165
16:12:26: step 20, duration = 4.140
16:13:07: step 30, duration = 4.143
16:13:49: step 40, duration = 4.053
16:14:33: step 50, duration = 4.624
16:15:15: step 60, duration = 4.112
16:15:58: step 70, duration = 4.198


KeyboardInterrupt: 

In [65]:
from tensorflow.contrib.slim.python.slim.nets import inception

In [66]:
inception??